In [1]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

![Original-ResNet-18-Architecture.png](Original-ResNet-18-Architecture.png)

In [2]:
orig_model = resnet18(pretrained=True)
orig_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
REDUCTION_RATES = [1, 2, 4, 8]
rank_map = {}

X = orig_model.conv1.weight.detach()
bias = orig_model.conv1.bias
print('Layer: conv1')
print('Original Shape:', X.shape)
X = X.reshape((X.shape[0], X.shape[1], -1))
print('Flattened Shape:', X.shape)
print('Parameters:', X.numel())
print('bias:', bias is not None)
# print('Feasible ranks <', X.numel()/sum(list(X.shape))/REDUCTION_RATE)
ranks = list(int(X.numel()/sum(list(X.shape))/rate) for rate in REDUCTION_RATES)
print('Feasible ranks = ', ranks)
print()
rank_map['conv1'] = ranks

for layer_name in ['layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 
                   'layer2.0.conv1', 'layer2.0.conv2', 'layer2.0.downsample', 'layer2.1.conv1', 'layer2.1.conv2',
                   'layer3.0.conv1', 'layer3.0.conv2', 'layer3.0.downsample', 'layer3.1.conv1', 'layer3.1.conv2',
                   'layer4.0.conv1', 'layer4.0.conv2', 'layer4.0.downsample', 'layer4.1.conv1', 'layer4.1.conv2']:
    lname, lidx, ltype = layer_name.split('.')
    lidx = int(lidx)
    layer = orig_model.__getattr__(lname)[lidx].__getattr__(ltype)
    if ltype == 'downsample': layer = layer[0]
    X = layer.weight.detach()
    bias = orig_model.conv1.bias
    print('Layer:', layer_name)
    print('Original Shape:', X.shape)
    X = X.reshape((X.shape[0], X.shape[1], -1))
    if ltype == 'downsample': X = X.reshape((X.shape[0], X.shape[1]))
    print('Flattened Shape:', X.shape)
    print('Parameters:', X.numel())
    print('bias:', bias is not None)
    ranks = list(int(X.numel()/sum(list(X.shape))/rate) for rate in REDUCTION_RATES)
    print('Feasible ranks = ', ranks)
    print()
    rank_map[layer_name] = ranks

X = orig_model.fc.weight.detach()
bias = orig_model.conv1.bias
print('Layer: fc')
print('Original Shape:', X.shape)
print('Parameters:', X.numel())
print('bias:', bias is not None)
# print('Feasible rank < ', X.numel()/sum(list(X.shape))/rate)
ranks = list(int(X.numel()/sum(list(X.shape))/rate) for rate in REDUCTION_RATES)
print('Feasible ranks = ', ranks)
rank_map['fc'] = ranks

Layer: conv1
Original Shape: torch.Size([64, 3, 7, 7])
Flattened Shape: torch.Size([64, 3, 49])
Parameters: 9408
bias: False
Feasible ranks =  [81, 40, 20, 10]

Layer: layer1.0.conv1
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Parameters: 36864
bias: False
Feasible ranks =  [269, 134, 67, 33]

Layer: layer1.0.conv2
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Parameters: 36864
bias: False
Feasible ranks =  [269, 134, 67, 33]

Layer: layer1.1.conv1
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Parameters: 36864
bias: False
Feasible ranks =  [269, 134, 67, 33]

Layer: layer1.1.conv2
Original Shape: torch.Size([64, 64, 3, 3])
Flattened Shape: torch.Size([64, 64, 9])
Parameters: 36864
bias: False
Feasible ranks =  [269, 134, 67, 33]

Layer: layer2.0.conv1
Original Shape: torch.Size([128, 64, 3, 3])
Flattened Shape: torch.Size([128, 64, 9])
Parameters: 73728
bias: False
Feasi

In [5]:
rank_map

{'conv1': [81, 40, 20, 10],
 'layer1.0.conv1': [269, 134, 67, 33],
 'layer1.0.conv2': [269, 134, 67, 33],
 'layer1.1.conv1': [269, 134, 67, 33],
 'layer1.1.conv2': [269, 134, 67, 33],
 'layer2.0.conv1': [366, 183, 91, 45],
 'layer2.0.conv2': [556, 278, 139, 69],
 'layer2.0.downsample': [42, 21, 10, 5],
 'layer2.1.conv1': [556, 278, 139, 69],
 'layer2.1.conv2': [556, 278, 139, 69],
 'layer3.0.conv1': [750, 375, 187, 93],
 'layer3.0.conv2': [1132, 566, 283, 141],
 'layer3.0.downsample': [85, 42, 21, 10],
 'layer3.1.conv1': [1132, 566, 283, 141],
 'layer3.1.conv2': [1132, 566, 283, 141],
 'layer4.0.conv1': [1518, 759, 379, 189],
 'layer4.0.conv2': [2283, 1141, 570, 285],
 'layer4.0.downsample': [170, 85, 42, 21],
 'layer4.1.conv1': [2283, 1141, 570, 285],
 'layer4.1.conv2': [2283, 1141, 570, 285],
 'fc': [338, 169, 84, 42]}